<a href="https://colab.research.google.com/github/vekteo/ASRT_ultra_fast_consolidation/blob/main/Offline_online_changes_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code for calculating offline vs. online changes between blocks (rapid consolidation online project)**

---

1. Load the file named *all_trials_last_repsonse_without_1000ms.csv* on OSF - this file contains all valid (first response) trials without the trials with more than 1000 ms response time
2. Hit *Run all*

Code by Teodóra Vékony https://github.com/vekteo

Lyon Neuroscience Research Center (CRNL), Université Claude Bernard Lyon 1

# Import Python packages

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import glob
import seaborn as sns
import matplotlib as plt

# Read datafiles

In [ ]:
# get data file names
filenames = glob.glob("*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding="utf8"))

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)

# Preprocessing


**1. Creating the bins**

In [ ]:
#define bins
df.loc[df['trial_number'] <= 16, 'bin'] = 1
df.loc[(df['trial_number'] > 16) & (df['trial_number'] <= 32), 'bin'] = 2
df.loc[(df['trial_number'] > 32) & (df['trial_number'] <= 48), 'bin'] = 3
df.loc[(df['trial_number'] > 48) & (df['trial_number'] <= 64), 'bin'] = 4
df.loc[(df['trial_number'] > 64) & (df['trial_number'] <= 80), 'bin'] = 5

**2. Exclude first trials, trills and repetitions**

In [ ]:
#drop X, T, R trials
indexNames = df[ df["triplet_type"] == "X" ].index
df.drop(indexNames, inplace=True)
indexNames2 = df[ df["triplet_type"] == "T" ].index
df.drop(indexNames2, inplace=True)
indexNames3 = df[ df["triplet_type"] == "R" ].index
df.drop(indexNames3, inplace=True)
df.head()

**3. Exclude incorrect trials**

In [ ]:
#drop incorrect trials
indexNames = df[ df["ACC"] == 0].index
df.drop(indexNames, inplace=True)
df.head()

# Calculating offline-online changes in statistical learning

**1. Change data to numeric**

In [ ]:
df['block'] = pd.to_numeric(df['block'])
df['bin'] = pd.to_numeric(df['bin'])

**2. Calculate H and L scores by bins**

In [ ]:
TL_RT_wide = df.copy()
TL_RT_wide= TL_RT_wide.pivot_table(index=['Participant Public ID','group'], columns=['block','bin','triplet_type'], values='rt', aggfunc=np.mean)
TL_RT_wide = TL_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5',6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                        11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20',
                                        21.0: 'block21', 22.0: 'block22', 23.0:'block23', 24.0:'block24', 25.0: 'block25'}, level=0)
TL_RT_wide = TL_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)

TL_RT_wide.columns = TL_RT_wide.columns.map('_'.join).str.strip('_')
TL_RT_wide.head()

**3. Calculate offline and online learning scores by bins**

*Offline: bin 1 of block minus bin 5 of previous block*

*Online: bin 5 of block minus bin 1 of same block*

In [ ]:
#calculate learning scores for each bin
for x in range(1,26):
  for y in [1,5]:
    TL_RT_wide["block"+str(x)+"_bin"+str(y)+'_TL'] = TL_RT_wide["block"+str(x)+"_bin"+str(y)+"_L"] - TL_RT_wide["block"+str(x)+"_bin"+str(y)+"_H"]

#calculate offline learning scores
for x in range(2,26):
    TL_RT_wide["block"+str(x)+'_offline'] = TL_RT_wide["block"+str(x)+"_bin1_TL"] - TL_RT_wide["block"+str(x-1)+"_bin5_TL"]

#calculate online learning scores
for x in range(1,26):
    TL_RT_wide["block"+str(x)+'_online'] = TL_RT_wide["block"+str(x)+"_bin5_TL"] - TL_RT_wide["block"+str(x)+"_bin1_TL"]

TL_RT_wide.head()

**4. Calculate overall offline and online learning scores**

*Averaging over the offline/online scores of each block*

In [ ]:
#calculate overall offline & online learning scores
TL_RT_wide["offline_TL"] = TL_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                       'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline','block17_offline', 'block18_offline','block19_offline', 'block20_offline',
                                       'block21_offline', 'block22_offline','block23_offline', 'block24_offline', 'block25_offline']].mean(axis=1)
                                       
TL_RT_wide["online_TL"] = TL_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                       'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online','block17_online', 'block18_online','block19_online', 'block20_online',
                                       'block21_online', 'block22_online','block23_online', 'block24_online','block25_online']].mean(axis=1)

TL_RT_wide.head()

**5. Save data in CSV**

In [ ]:
TL_RT_wide.to_csv("TL_RT_online_offline.csv",index=True)
files.download("TL_RT_online_offline.csv")

# Calculating offline-online changes in general skill learning

**1. Calculate general RT scores by bins**

In [ ]:
GS_RT_wide = df.copy()
GS_RT_wide= GS_RT_wide.pivot_table(index=['Participant Public ID','group'], columns=['block','bin'], values='rt', aggfunc=np.mean)
GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5',6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                        11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20',
                                        21.0: 'block21', 22.0: 'block22', 23.0:'block23', 24.0:'block24', 25.0: 'block25'}, level=0)
GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)

GS_RT_wide.columns = GS_RT_wide.columns.map('_'.join).str.strip('_')
GS_RT_wide.head()

**2. Calculate offline and online learning scores by bins**


*Offline: bin 1 of block minus bin 5 of previous block*

*Online: bin 5 of block minus bin 1 of same block*

In [ ]:
#calculate offline learning scores
for x in range(2,26):
    GS_RT_wide["block"+str(x)+'_offline'] = GS_RT_wide["block"+str(x)+"_bin1"] - GS_RT_wide["block"+str(x-1)+"_bin5"]

#calculate online learning scores
for x in range(1,26):
    GS_RT_wide["block"+str(x)+'_online'] = GS_RT_wide["block"+str(x)+"_bin5"] - GS_RT_wide["block"+str(x)+"_bin1"]

GS_RT_wide.head()

**3. Calculate overall offline and online learning scores**

*Averaging over the offline/online scores of each block*

In [ ]:
#calculate overall offline & online learning scores
GS_RT_wide["offline_GS"] = GS_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                       'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline','block17_offline', 'block18_offline','block19_offline', 'block20_offline',
                                       'block21_offline', 'block22_offline','block23_offline', 'block24_offline', 'block25_offline']].mean(axis=1)
                                       
GS_RT_wide["online_GS"] = GS_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                       'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online','block17_online', 'block18_online','block19_online', 'block20_online',
                                       'block21_online', 'block22_online','block23_online', 'block24_online','block25_online']].mean(axis=1)

GS_RT_wide.head()

**4. Save the data to CSV**

In [ ]:
GS_RT_wide.to_csv("GS_RT_online_offline.csv",index=True)
files.download("GS_RT_online_offline.csv")